In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd
import os
from glob import glob

# Extract SWE and SST data for Tibetan Plateau and NINO3 region respectively

In [6]:
hist_source = ['ACCESS-CM2', 'ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CESM2', 'CESM2-WACCM', 'CIESM', 'CNRM-CM6-1', 'CanESM5', 'CanESM5-1', 'E3SM-1-0', 'E3SM-2-0',
'EC-Earth3','EC-Earth3-AerChem','EC-Earth3-CC','EC-Earth3-Veg','GFDL-CM4','GFDL-ESM4','GISS-E2-1-G','GISS-E2-1-H','GISS-E2-2-G','HadGEM3-GC31-LL','IPSL-CM5A2-INCA',
'IPSL-CM6A-LR','KIOST-ESM','MIROC6','MPI-ESM-1-2-HAM','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0','NorESM2-LM','NorESM2-MM','TaiESM1']
SSP245_source = ['ACCESS-CM2', 'ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CESM2', 'CESM2-WACCM', 'CIESM', 'CNRM-CM6-1', 'CanESM5', 'CanESM5-1',
'EC-Earth3','EC-Earth3-CC','EC-Earth3-Veg','GFDL-CM4','GFDL-ESM4','GISS-E2-1-G','GISS-E2-1-H','GISS-E2-2-G','HadGEM3-GC31-LL',
'IPSL-CM6A-LR','KIOST-ESM','MIROC6','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0','NorESM2-LM','NorESM2-MM','TaiESM1']
SSP370_source = ['E3SM-1-0', 'E3SM-2-0', 'EC-Earth3-AerChem','IPSL-CM5A2-INCA','MPI-ESM-1-2-HAM']
NAT_source = ['ACCESS-CM2', 'ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CESM2', 'CNRM-CM6-1', 'CanESM5', 'E3SM-2-0',
'GFDL-CM4','GFDL-ESM4','GISS-E2-1-G','HadGEM3-GC31-LL', 'IPSL-CM6A-LR','MIROC6', 'MRI-ESM2-0','NorESM2-LM']

hist_variant = {
    'ACCESS-CM2':'r1i1p1f1', 'ACCESS-ESM1-5':'r1i1p1f1', 'BCC-CSM2-MR':'r1i1p1f1', 'CESM2':'r10i1p1f1', 'CESM2-WACCM':'r1i1p1f1', 'CIESM':'r1i1p1f1', 
    'CNRM-CM6-1':'r1i1p1f2', 'CanESM5':'r1i1p1f1', 'CanESM5-1':'r1i1p1f1', 'E3SM-1-0':'r10i2p2f1', 'E3SM-2-0':'r1i1p1f1','EC-Earth3':'r1i1p1f1',
    'EC-Earth3-AerChem':'r1i1p1f1','EC-Earth3-CC':'r1i1p1f1','EC-Earth3-Veg':'r1i1p1f1','GFDL-CM4':'r1i1p1f1','GFDL-ESM4':'r1i1p1f1',
    'GISS-E2-1-G':'r101i1p1f1','GISS-E2-1-H':'r1i1p1f2','GISS-E2-2-G':'r1i1p3f1','HadGEM3-GC31-LL':'r1i1p1f3','IPSL-CM5A2-INCA':'r1i1p1f1',
    'IPSL-CM6A-LR':'r1i1p1f1','KIOST-ESM':'r1i1p1f1','MIROC6':'r1i1p1f1','MPI-ESM-1-2-HAM':'r1i1p1f1','MPI-ESM1-2-HR':'r1i1p1f1','MPI-ESM1-2-LR':'r1i1p1f1',
    'MRI-ESM2-0':'r1i1p1f1','NorESM2-LM':'r1i1p1f1','NorESM2-MM':'r1i1p1f1','TaiESM1':'r1i1p1f1'
}
SSP245_variant = {
    'ACCESS-CM2':'r1i1p1f1', 'ACCESS-ESM1-5':'r1i1p1f1', 'BCC-CSM2-MR':'r1i1p1f1', 'CESM2':'r10i1p1f1', 'CESM2-WACCM':'r1i1p1f1', 'CIESM':'r1i1p1f1', 
    'CNRM-CM6-1':'r1i1p1f2', 'CanESM5':'r1i1p1f1', 'CanESM5-1':'r1i1p1f1', 'EC-Earth3':'r1i1p1f1',
    'EC-Earth3-CC':'r1i1p1f1','EC-Earth3-Veg':'r1i1p1f1','GFDL-CM4':'r1i1p1f1','GFDL-ESM4':'r1i1p1f1',
    'GISS-E2-1-G':'r101i1p1f1','GISS-E2-1-H':'r1i1p1f2','GISS-E2-2-G':'r1i1p3f1','HadGEM3-GC31-LL':'r1i1p1f3',
    'IPSL-CM6A-LR':'r1i1p1f1','KIOST-ESM':'r1i1p1f1','MIROC6':'r1i1p1f1','MPI-ESM1-2-HR':'r1i1p1f1','MPI-ESM1-2-LR':'r1i1p1f1',
    'MRI-ESM2-0':'r1i1p1f1','NorESM2-LM':'r1i1p1f1','NorESM2-MM':'r1i1p1f1','TaiESM1':'r1i1p1f1'
}
SSP370_variant = {
    'E3SM-1-0':'r10i2p2f1', 'E3SM-2-0':'r1i1p1f1', 'EC-Earth3-AerChem':'r1i1p1f1','IPSL-CM5A2-INCA':'r1i1p1f1','MPI-ESM-1-2-HAM':'r1i1p1f1'
}
NAT_variant = {
    'ACCESS-CM2':'r1i1p1f1', 'ACCESS-ESM1-5':'r1i1p1f1', 'BCC-CSM2-MR':'r1i1p1f1', 'CESM2':'r1i1p1f1', 'CNRM-CM6-1':'r1i1p1f2', 'CanESM5':'r1i1p1f1',
    'E3SM-2-0':'r1i1p1f1','GFDL-CM4':'r1i1p1f1','GFDL-ESM4':'r1i1p1f1','GISS-E2-1-G':'r1i1p1f3','HadGEM3-GC31-LL':'r1i1p1f3', 
    'IPSL-CM6A-LR':'r1i1p1f1','MIROC6':'r1i1p1f1','MRI-ESM2-0':'r1i1p1f1','NorESM2-LM':'r1i1p1f1'
}

paths = {
    'hist':'/Volumes/Seagate_ZQ2/SWE_datasets/rawData/CMIP6/historical/regrid',
    'SSP245':'/Volumes/Seagate_ZQ2/SWE_datasets/rawData/CMIP6/ssp245/regrid',
    'SSP370':'/Volumes/Seagate_ZQ2/SWE_datasets/rawData/CMIP6/ssp370/regrid',
    'NAT':'/Volumes/Seagate_ZQ2/SWE_datasets/rawData/CMIP6/NAT/regrid'
}

paths_2 = {
    'hist':'/Volumes/Seagate_ZQ2/SWE_datasets/rawData/CMIP6/historical',
    'SSP245':'/Volumes/Seagate_ZQ2/SWE_datasets/rawData/CMIP6/ssp245',
    'SSP370':'/Volumes/Seagate_ZQ2/SWE_datasets/rawData/CMIP6/ssp370',
    'NAT':'/Volumes/Seagate_ZQ2/SWE_datasets/rawData/CMIP6/NAT'
}

TP_range = {'lon1':66, 'lon2':108, 'lat1':22, 'lat2':46}
Nino3_region_range = {'lon1':210, 'lon2':270, 'lat1':-5, 'lat2':5}

In [17]:
# select SWE data of the TP region for historical 
for s in hist_source:
    snw_path_s = glob(paths['hist'] + '/' + 'snw' + '*' + s + '_*' + hist_variant[s] + '*.nc')[0]
    snw_s = xr.open_dataset(snw_path_s).sel(lon=slice(TP_range['lon1'], TP_range['lon2']), lat=slice(TP_range['lat1'], TP_range['lat2']))
    if s in SSP245_source:
        snw_path_s = glob(paths['SSP245'] + '/' + 'snw' + '*' + s + '_*' + SSP245_variant[s] + '*.nc')[0]
        snw_s = xr.concat([snw_s, xr.open_dataset(snw_path_s).sel(lon=slice(TP_range['lon1'], TP_range['lon2']), lat=slice(TP_range['lat1'], TP_range['lat2']))], dim='time')
    elif s in SSP370_source:
        snw_path_s = glob(paths['SSP370'] + '/' + 'snw' + '*' + s + '_*' + SSP370_variant[s] + '*.nc')[0]
        snw_s = xr.concat([snw_s, xr.open_dataset(snw_path_s).sel(lon=slice(TP_range['lon1'], TP_range['lon2']), lat=slice(TP_range['lat1'], TP_range['lat2']))], dim='time')
    # select from 1950~2025
    snw_s = snw_s.sel(time=slice('1950', '2025'))
    # save to file
    snw_s.to_netcdf('/Volumes/Seagate_ZQ2/SWE_datasets/rawData/CMIP6/HIST/' + 'snw' + '_TP_' + s + '_' + hist_variant[s] + '_1950-2025.nc')

In [13]:
# select pr data of the TP region for historical 
for s in hist_source:
    if s == 'GFDL-CM4':
        continue
    pr_path_s = glob(paths_2['hist'] + '/' + 'pr' + '*' + s + '_*' + hist_variant[s] + '*.nc')[0]
    pr_s = xr.open_dataset(pr_path_s)['pr'].sel(lon=slice(TP_range['lon1'], TP_range['lon2']), lat=slice(TP_range['lat1'], TP_range['lat2']))
    if s in SSP245_source:
        pr_path_s = glob(paths_2['SSP245'] + '/' + 'pr' + '*' + s + '_*' + SSP245_variant[s] + '*.nc')[0]
        pr_s = xr.concat([pr_s, xr.open_dataset(pr_path_s)['pr'].sel(lon=slice(TP_range['lon1'], TP_range['lon2']), lat=slice(TP_range['lat1'], TP_range['lat2']))], dim='time')
    elif s in SSP370_source:
        pr_path_s = glob(paths_2['SSP370'] + '/' + 'pr' + '*' + s + '_*' + SSP370_variant[s] + '*.nc')[0]
        pr_s = xr.concat([pr_s, xr.open_dataset(pr_path_s)['pr'].sel(lon=slice(TP_range['lon1'], TP_range['lon2']), lat=slice(TP_range['lat1'], TP_range['lat2']))], dim='time')
    # select from 1950~2025
    pr_s = pr_s.sel(time=slice('1950', '2025'))
    # save to file
    pr_s.to_netcdf('/Volumes/Seagate_ZQ2/SWE_datasets/rawData/CMIP6/HIST/' + 'pr' + '_TP_' + s + '_' + hist_variant[s] + '_1950-2025.nc')

/Users/zeqinhuang/miniforge3/envs/threeDMap/lib/python3.11/site-packages/xarray/conventions.py:551: SerializationWarning: variable 'pr' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


In [14]:
# select tas data of the TP region for historical 
for s in hist_source:
    if s == 'GFDL-CM4':
        continue
    tas_path_s = glob(paths_2['hist'] + '/' + 'tas' + '*' + s + '_*' + hist_variant[s] + '*.nc')[0]
    tas_s = xr.open_dataset(tas_path_s)['tas'].sel(lon=slice(TP_range['lon1'], TP_range['lon2']), lat=slice(TP_range['lat1'], TP_range['lat2']))
    if s in SSP245_source:
        tas_path_s = glob(paths_2['SSP245'] + '/' + 'tas' + '*' + s + '_*' + SSP245_variant[s] + '*.nc')[0]
        tas_s = xr.concat([tas_s, xr.open_dataset(tas_path_s)['tas'].sel(lon=slice(TP_range['lon1'], TP_range['lon2']), lat=slice(TP_range['lat1'], TP_range['lat2']))], dim='time')
    elif s in SSP370_source:
        tas_path_s = glob(paths_2['SSP370'] + '/' + 'tas' + '*' + s + '_*' + SSP370_variant[s] + '*.nc')[0]
        tas_s = xr.concat([tas_s, xr.open_dataset(tas_path_s)['tas'].sel(lon=slice(TP_range['lon1'], TP_range['lon2']), lat=slice(TP_range['lat1'], TP_range['lat2']))], dim='time')
    # select from 1950~2025
    tas_s = tas_s.sel(time=slice('1950', '2025'))
    # save to file
    tas_s.to_netcdf('/Volumes/Seagate_ZQ2/SWE_datasets/rawData/CMIP6/HIST/' + 'tas' + '_TP_' + s + '_' + hist_variant[s] + '_1950-2025.nc')

/Users/zeqinhuang/miniforge3/envs/threeDMap/lib/python3.11/site-packages/xarray/conventions.py:551: SerializationWarning: variable 'tas' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


In [4]:
# select SWE data of the TP region for historical 
for s in ['NorESM2-LM']:
    snw_path_s = glob(paths['hist'] + '/' + 'snw' + '*' + s + '_*' + hist_variant[s] + '*.nc')[0]
    snw_s = xr.open_dataset(snw_path_s).sel(lon=slice(TP_range['lon1'], TP_range['lon2']), lat=slice(TP_range['lat1'], TP_range['lat2']))
    if s in SSP245_source:
        snw_path_s = glob(paths['SSP245'] + '/' + 'snw' + '*' + s + '_*' + SSP245_variant[s] + '*.nc')[0]
        snw_s = xr.concat([snw_s, xr.open_dataset(snw_path_s).sel(lon=slice(TP_range['lon1'], TP_range['lon2']), lat=slice(TP_range['lat1'], TP_range['lat2']))], dim='time')
    elif s in SSP370_source:
        snw_path_s = glob(paths['SSP370'] + '/' + 'snw' + '*' + s + '_*' + SSP370_variant[s] + '*.nc')[0]
        snw_s = xr.concat([snw_s, xr.open_dataset(snw_path_s).sel(lon=slice(TP_range['lon1'], TP_range['lon2']), lat=slice(TP_range['lat1'], TP_range['lat2']))], dim='time')
    # select from 1950~2025
    snw_s = snw_s.sel(time=slice('1950', '2025'))
    # save to file
    snw_s.to_netcdf('/Volumes/Seagate_ZQ2/SWE_datasets/rawData/CMIP6/HIST/' + 'snw' + '_TP_' + s + '_' + hist_variant[s] + '_1950-2025.nc')

In [73]:
# select SWE data of the TP region for NAT scenario
for s in NAT_source:
    snw_path_s = glob(paths['NAT'] + '/' + 'snw' + '*' + s + '_*' + NAT_variant[s] + '*.nc')[0]
    snw_s = xr.open_dataset(snw_path_s).sel(lon=slice(TP_range['lon1'], TP_range['lon2']), lat=slice(TP_range['lat1'], TP_range['lat2']))
    last_year = snw_s['time'][-1].dt.year.values
    # select from 1950~last_year
    snw_s = snw_s.sel(time=slice('1950', str(last_year)))
    # save to file
    snw_s.to_netcdf('/Volumes/Seagate_ZQ2/SWE_datasets/rawData/CMIP6/NAT/' + 'snw' + '_TP_' + s + '_' + NAT_variant[s] + '_1950-' + str(last_year) + '.nc')

In [15]:
# select pr data of the TP region for NAT scenario
for s in NAT_source:
    pr_path_s = glob(paths_2['NAT'] + '/' + 'pr' + '*' + s + '_*' + NAT_variant[s] + '*.nc')[0]
    pr_s = xr.open_dataset(pr_path_s)['pr'].sel(lon=slice(TP_range['lon1'], TP_range['lon2']), lat=slice(TP_range['lat1'], TP_range['lat2']))
    last_year = pr_s['time'][-1].dt.year.values
    # select from 1950~last_year
    pr_s = pr_s.sel(time=slice('1950', str(last_year)))
    # save to file
    pr_s.to_netcdf('/Volumes/Seagate_ZQ2/SWE_datasets/rawData/CMIP6/NAT/' + 'pr' + '_TP_' + s + '_' + NAT_variant[s] + '_1950-' + str(last_year) + '.nc')

In [16]:
# select tas data of the TP region for NAT scenario
for s in NAT_source:
    tas_path_s = glob(paths_2['NAT'] + '/' + 'tas' + '*' + s + '_*' + NAT_variant[s] + '*.nc')[0]
    tas_s = xr.open_dataset(tas_path_s)['tas'].sel(lon=slice(TP_range['lon1'], TP_range['lon2']), lat=slice(TP_range['lat1'], TP_range['lat2']))
    last_year = tas_s['time'][-1].dt.year.values
    # select from 1950~last_year
    tas_s = tas_s.sel(time=slice('1950', str(last_year)))
    # save to file
    tas_s.to_netcdf('/Volumes/Seagate_ZQ2/SWE_datasets/rawData/CMIP6/NAT/' + 'tas' + '_TP_' + s + '_' + NAT_variant[s] + '_1950-' + str(last_year) + '.nc')

In [20]:
# select tos data of the Nino3 region for historical
for s in hist_source:
    tos_path_s = glob(paths['hist'] + '/' + 'tos' + '*' + s + '_*' + hist_variant[s] + '*.nc')[0]
    tos_s = xr.open_dataset(tos_path_s).sel(lon=slice(Nino3_region_range['lon1'], Nino3_region_range['lon2']), lat=slice(Nino3_region_range['lat1'], Nino3_region_range['lat2']))
    if s in SSP245_source:
        tos_path_s = glob(paths['SSP245'] + '/' + 'tos' + '*' + s + '_*' + SSP245_variant[s] + '*.nc')[0]
        tos_s = xr.concat([tos_s, xr.open_dataset(tos_path_s).sel(lon=slice(Nino3_region_range['lon1'], Nino3_region_range['lon2']), lat=slice(Nino3_region_range['lat1'], Nino3_region_range['lat2']))], dim='time')
    elif s in SSP370_source:
        tos_path_s = glob(paths['SSP370'] + '/' + 'tos' + '*' + s + '_*' + SSP370_variant[s] + '*.nc')[0]
        tos_s = xr.concat([tos_s, xr.open_dataset(tos_path_s).sel(lon=slice(Nino3_region_range['lon1'], Nino3_region_range['lon2']), lat=slice(Nino3_region_range['lat1'], Nino3_region_range['lat2']))], dim='time')
    # select from 1950~2025
    tos_s = tos_s.sel(time=slice('1950', '2025'))
    # save to file
    tos_s.to_netcdf('/Volumes/Seagate_ZQ2/SWE_datasets/rawData/CMIP6/HIST/' + 'tos' + '_Nino3_' + s + '_' + hist_variant[s] + '_1950-2025.nc')

In [74]:
# select tos data of the Nino3 region for NAT scenario
for s in NAT_source:
    tos_path_s = glob(paths['NAT'] + '/' + 'tos' + '*' + s + '_*' + NAT_variant[s] + '*.nc')[0]
    tos_s = xr.open_dataset(tos_path_s).sel(lon=slice(Nino3_region_range['lon1'], Nino3_region_range['lon2']), lat=slice(Nino3_region_range['lat1'], Nino3_region_range['lat2']))
    last_year = tos_s['time'][-1].dt.year.values
    # select from 1950~last_year
    tos_s = tos_s.sel(time=slice('1950', str(last_year)))
    # save to file
    tos_s.to_netcdf('/Volumes/Seagate_ZQ2/SWE_datasets/rawData/CMIP6/NAT/' + 'tos' + '_Nino3_' + s + '_' + NAT_variant[s] + '_1950-' + str(last_year) + '.nc')

# Nino X index computation
https://climatedataguide.ucar.edu/climate-data/nino-sst-indices-nino-12-3-34-4-oni-and-tni

Nino X Index computation: (a) Compute area averaged total SST from Niño X region; (b) Compute monthly climatology (e.g., 1950-1979) for area averaged total SST from Niño X region, and subtract climatology from area averaged total SST time series to obtain anomalies; (c) Smooth the anomalies with a 5-month running mean; (d) Normalize the smoothed values by its standard deviation over the climatological period. 

In [32]:
# define a function to calculate nino3 index
# Nino X Index computation: (a) Compute area averaged total SST from Niño X region; (b) Compute monthly climatology (e.g., 1950-1979) for 
# area averaged total SST from Niño X region, and subtract climatology from area averaged total SST time series to obtain anomalies; 
# (c) Smooth the anomalies with a 5-month running mean; (d) Normalize the smoothed values by its standard deviation over the climatological period. 
def detrend(x, axis=0):
    # remove the long-term trend
    x = x - np.polyval(np.polyfit(np.arange(len(x)), x, 1), np.arange(len(x)))
    return x

def nino3_index(tos):
    # compute area averaged total SST from Niño X region
    tos_nino3 = tos.mean(dim=['lon', 'lat'])
    # compute monthly climatology (e.g., 1950-1979) for area averaged total SST from Niño X region
    tos_nino3_clim = tos_nino3.sel(time=slice('1950', '1979')).groupby('time.month').mean(dim='time')
    # subtract climatology from area averaged total SST time series to obtain anomalies
    tos_nino3_anom = tos_nino3.groupby('time.month') - tos_nino3_clim
    # remove the long-term trend
    tos_nino3_anom = xr.apply_ufunc(detrend, tos_nino3_anom, kwargs={'axis': 0}).where(~tos_nino3_anom.isnull())
    # smooth the anomalies with a 5-month running mean
    tos_nino3_anom_smooth = tos_nino3_anom.rolling(time=5, center=True).mean()
    # normalize the smoothed values by its standard deviation over the climatological period
    tos_nino3_anom_smooth_std = tos_nino3_anom_smooth.sel(time=slice('1950', '1979')).std(dim='time')
    tos_nino3_index = tos_nino3_anom_smooth / tos_nino3_anom_smooth_std
    return tos_nino3_index

In [37]:
# calculate nino3 index of each model for historical
tos_nino3_index_all = pd.DataFrame()
for s in hist_source:
    tos = xr.open_dataset('/Volumes/Seagate_ZQ2/SWE_datasets/rawData/CMIP6/HIST/' + 'tos' + '_Nino3_' + s + '_' + hist_variant[s] + '_1950-2025.nc')
    tos_nino3_index = nino3_index(tos['tos'])
    tos_nino3_index_all[s] = tos_nino3_index

tos_nino3_index_all.index = pd.date_range('1950-01', '2026-01', freq='M')
tos_nino3_index_all.to_csv('/Volumes/Seagate_ZQ2/SWE_datasets/rawData/CMIP6/HIST/' + 'tos' + '_Nino3_index_1950-2025.csv')

In [85]:
# calculate nino3 index of each model for NAT
tos_nino3_index_all = pd.DataFrame()
for s in NAT_source:
    tos = xr.open_dataset(glob('/Volumes/Seagate_ZQ2/SWE_datasets/rawData/CMIP6/NAT/' + 'tos' + '_Nino3_' + s + '_' + NAT_variant[s] + '_1950-*.nc')[0])
    last_year = tos['time'][-1].dt.year.values
    tos_nino3_index = nino3_index(tos['tos'])
    tos_nino3_index = tos_nino3_index.to_dataframe()
    if s == 'GISS-E2-1-G':
        tos_nino3_index.index = pd.date_range('1951-01', str(last_year+1)+'-01', freq='M')
    else:
        tos_nino3_index.index = pd.date_range('1950-01', str(last_year+1)+'-01', freq='M')
    tos_nino3_index_all[s] = tos_nino3_index['tos']

tos_nino3_index_all.to_csv('/Volumes/Seagate_ZQ2/SWE_datasets/rawData/CMIP6/NAT/' + 'tos' + '_Nino3_index_1950-2020.csv')